In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import analyze_utils
import dvu
import re
dvu.set_style()

import sys
import iprompt.data

def add_trailing_space(s):
    if not s.endswith(' '):
        return s + ' '
    return s

def clean_task_name(s):
    s = s.replace('_', ' ')
    if s.startswith('task'): # anli
        s = s[s.index(' '):]
    return s

/Users/chandan/rules/interpretable-autoprompting/.autoprompt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Print table

In [3]:
prompts_all = pd.read_pickle('../results/generalization_acc/prompts_all.pkl')

In [7]:
p = prompts_all.rename(columns={
    'manual': 'Human-written prompt',
    'iprompt': 'iPrompt',
    'autoprompt': 'AutoPrompt',
    'suffix': 'Average suffix',
})
p.index = pd.Series(p.index).apply(clean_task_name).values
with pd.option_context("max_colwidth", 1000):
    print(p.to_latex().replace('\_peixian\_equity\_evaluation\_corpus', '').replace('�', ''))

\begin{tabular}{lllll}
\toprule
{} &                                                                                                                                                                                                                                                                                                                                                                                                                                                                       Human-written prompt &                                          AutoPrompt &                                    iPrompt &                           Average suffix \\
\midrule
 identify typo verification                         &  The given sentence contains a typo which could be one of the following four types: (1) swapped letters of a word e.g. 'niec' is a typo of the word 'nice'. (2) missing letter in a word e.g. 'nic' is a typo of the word 'nice'. (3) extra letter in a word e.g. 'nicce' is a typo of t

/var/folders/01/fl5q4n5547v1g4z0hw_v_mrc0000gn/T/ipykernel_73767/3459251185.py:9: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(p.to_latex().replace('\_peixian\_equity\_evaluation\_corpus', '').replace('�', ''))


In [8]:

showcase = ['task107_splash_question_to_sql',
'task1147_country_currency',
'task1191_food_veg_nonveg',
       'task1509_evalution_antonyms', 'add_two',
       'multiply_two','subtract_two', 'max_two', 'first_two',
       'square_one', 'double_one']
p = prompts_all[['manual', 'iprompt', 'autoprompt']].rename(columns={
    'manual': 'Human-written prompt',
    'iprompt': 'iPrompt',
    'autoprompt': 'AutoPrompt',
}).loc[showcase]

p.index = pd.Series(p.index).apply(clean_task_name).values
with pd.option_context("max_colwidth", 1000):
    print(p.to_latex().replace('\_peixian\_equity\_evaluation\_corpus', '').replace('�', ''))

\begin{tabular}{llll}
\toprule
{} &                                                                                                                           Human-written prompt &                                    iPrompt &                                        AutoPrompt \\
\midrule
 splash question to sql &                                                                            Generate an SQL statement from a question asking for certain data.  &            Write an SQL to produce output  &  ributed grandfatherExceptionappropri intent Lara \\
 country currency       &                                                        You are given a country name and you need to return the currency of the given country.  &            Select currency code for a new  &                    renciesthethe Dmitrythe mortg  \\
 food veg nonveg        &                                                                                 Return whether the input food dish is vegetarian (yes or no).  &    

/var/folders/01/fl5q4n5547v1g4z0hw_v_mrc0000gn/T/ipykernel_73767/951248674.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(p.to_latex().replace('\_peixian\_equity\_evaluation\_corpus', '').replace('�', ''))


## Load prompts from sweep results

In [41]:
prompts_file = '../results/generalization_acc/math_prompts_with_accs_EleutherAI__gpt-j-6B.pkl'
df = pkl.load(open(prompts_file, 'rb'))

In [46]:
d = df
task_names = d.task_name.unique()
r = defaultdict(list)
for task_name in task_names:
    d = df
    d1 = d[d['legend'] == 'Human-written prompt']
    r['Human-written prompt'].append(d1[d1.task_name == task_name].iloc[0]['prompt_actual'])


    d = df[(df['legend'] == 'iPrompt (6 tokens)')]
    d = d[(d.n_shots == 1) * (d.max_digit==10) * (d.model_name == 'model_6b')]
    r['iPrompt (6 tokens)'].append(d[d.task_name == task_name].iloc[0]['prefixes'])

    d = df[(df['legend'] == 'AutoPrompt (6 tokens)')]
    d = d[(d.n_shots == 1) * (d.max_digit==10) * (d.model_name == 'model_6b')]
    r['AutoPrompt (6 tokens)'].append(d[d.task_name == task_name].iloc[0]['prefixes'])

    d

rf = pd.DataFrame.from_dict(r) #, index=task_names)
rf.index = task_names
rf

,Human-written prompt,iPrompt (6 tokens),AutoPrompt (6 tokens)
add_two,Return the sum of the inputs.,Write a function int add(,addedthe +the use worked
multiply_two,Return the product of the inputs.,When you multiply two (,multiplythethe �the Multiple
divide_two,Return the quotient of the inputs.,"n / N,where we",Kaplan MAG comprisingthe quarterly disproportion
subtract_two,Return the difference of the inputs.,If n > m then subtract,opposably exactly subtractFor YEAR
max_two,Return the maximum of the inputs.,Which number has a bigger value,NumberthetheJusticeJaDefault
first_two,Return the first of the inputs.,The first digit of both values,greater name sorting indiscrim to numbers
square_one,Square the input to get the output.,Write a function that calculates square,multiplythe hypot Norttheirl
exp_one,Exponentiate the input to get the output.,Use BigInteger and double (,Smythe webpage fle clin McA
double_one,"Given an input x, return 2*x.",write a function called double that,ADDthe introducedpareat contraceptives
fibonacci_one,"Given an input x, return the xth fibonacci num...",how to prove an algorithm correctness,betweenthe made one uped


Print to latex

In [56]:
s = rf.to_latex()
print(s)

\begin{tabular}{llll}
\toprule
{} &                                             manual &                                    iprompt &                                         autoprompt \\
\midrule
add\_two       &                     Return the sum of the inputs.  &                 Write a function int add(  &                          addedthe +the use worked  \\
multiply\_two  &                 Return the product of the inputs.  &                   When you multiply two (  &                      multiplythethe �the Multiple  \\
divide\_two    &                Return the quotient of the inputs.  &                            n / N,where we  &   Kaplan MAG comprisingthe quarterly disproport... \\
subtract\_two  &              Return the difference of the inputs.  &                    If n > m then subtract  &                opposably exactly subtractFor YEAR  \\
max\_two       &                 Return the maximum of the inputs.  &           Which number has a bigger value  &             

/tmp/ipykernel_1961039/3591879776.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  s = rf.to_latex()


# Save out the prompts

In [48]:
for k in rf.columns:
    rf[k] = rf[k].apply(add_trailing_space)
rf = rf.rename(columns={
    'Human-written prompt': 'manual',
    'iPrompt (6 tokens)': 'iprompt',
    'AutoPrompt (6 tokens)': 'autoprompt',
})
rf.to_pickle('../results/generalization_acc/prompts.pkl')

In [49]:
rf

,manual,iprompt,autoprompt
add_two,Return the sum of the inputs.,Write a function int add(,addedthe +the use worked
multiply_two,Return the product of the inputs.,When you multiply two (,multiplythethe �the Multiple
divide_two,Return the quotient of the inputs.,"n / N,where we",Kaplan MAG comprisingthe quarterly disproport...
subtract_two,Return the difference of the inputs.,If n > m then subtract,opposably exactly subtractFor YEAR
max_two,Return the maximum of the inputs.,Which number has a bigger value,NumberthetheJusticeJaDefault
first_two,Return the first of the inputs.,The first digit of both values,greater name sorting indiscrim to numbers
square_one,Square the input to get the output.,Write a function that calculates square,multiplythe hypot Norttheirl
exp_one,Exponentiate the input to get the output.,Use BigInteger and double (,Smythe webpage fle clin McA
double_one,"Given an input x, return 2*x.",write a function called double that,ADDthe introducedpareat contraceptives
fibonacci_one,"Given an input x, return the xth fibonacci num...",how to prove an algorithm correctness,betweenthe made one uped


## Add ANLI prompts

In [86]:
prompts_anli_saved = pd.read_pickle('../results/generalization_acc/anli_prompts__n_shots_1__num_learned_tokens__6.pkl')
prompts_anli = pd.DataFrame()
prompts_anli['autoprompt'] = prompts_anli_saved[('prefixes', 'autoprompt')].astype(str).apply(add_trailing_space)
prompts_anli['iprompt'] = prompts_anli_saved[('prefixes', 'iprompt')].astype(str).apply(add_trailing_space)

def get_descr(task_name):
    class fake_args:
        template_num_task_phrasing = 0
        max_dset_size = 1000
        max_digit = 10
        seed = 1
        train_split_frac = 0.75

        # these will be varied
        n_shots = 1
    args = fake_args()
    dset, check_answer_func, descr = iprompt.data.get_data(args, task_name=task_name)
    return descr

prompts_anli = prompts_anli.reset_index()
prompts_anli.insert(0, 'manual', prompts_anli['task_name'].apply(get_descr))
prompts_anli.index = prompts_anli['task_name'].values
prompts_anli =  prompts_anli.drop(columns=['task_name'])

In [91]:
prompts_all = pd.concat([prompts_anli, rf])
# prompts_all.to_pickle('../results/generalization_acc/prompts_all.pkl')